# Unit 2: Popularity Recommendations

In this section we build a recommender that sorts items by popularity as of the number of ratings they received. As a result we return the $N$ most popular items as recommendations.

In [1]:
from typing import Dict, List

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

In [2]:
# `Dataset` is just a wrapper for the MovieLens training data
from recsys_training.data import Dataset, genres

In [3]:
ml100k_ratings_filepath = '../../data/raw/ml-100k/u.data'
ml100k_item_filepath = '../../data/raw/ml-100k/u.item'

## Load Data

We load the dataset with 100,000 ratings and split it $4:1$ into train and test set.

(**Remark**: We do not focus on proper hyperparameter search within this tutorial and therefore do not generate a separate validation dataset)

In [4]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(train_size=0.8, seed=42)

In [5]:
items = pd.read_csv(ml100k_item_filepath, sep='|', header=None,
                    names=['item', 'title', 'release', 'video_release', 'imdb_url']+genres,
                    engine='python')

In [6]:
data.train_ratings

,user,item,rating,timestamp
75721,877,381,4,882677345
80184,815,602,3,878694269
19864,94,431,4,891721716
76699,416,875,2,876696938
92991,500,182,2,883873556
...,...,...,...,...
6306,92,48,4,875653307
51901,671,54,3,884035173
6028,1,96,5,875072716
58507,82,169,4,878769442


In [7]:
data.test_ratings

,user,item,rating,timestamp
98877,907,143,5,880159982
24044,371,210,4,880435313
48435,218,42,4,877488546
72179,829,170,4,881698933
80645,733,277,1,879536523
...,...,...,...,...
6265,216,231,2,880245109
54886,343,276,5,876403078
76820,437,475,3,880140288
860,284,322,3,885329671


Build a Mapping from user id to its item ratings. We will need this later.

In [8]:
user_ratings = data.get_user_ratings()

Show up to 20 user ratings for the first user

In [9]:
user = 1
list(user_ratings[user].items())[:20]

[(233, 2.0),
 (159, 3.0),
 (238, 4.0),
 (100, 5.0),
 (63, 2.0),
 (192, 4.0),
 (181, 5.0),
 (6, 5.0),
 (103, 1.0),
 (156, 4.0),
 (259, 1.0),
 (258, 5.0),
 (31, 3.0),
 (155, 2.0),
 (142, 2.0),
 (38, 3.0),
 (206, 4.0),
 (217, 3.0),
 (263, 1.0),
 (193, 4.0)]

## Popularity Ranking

How do we define _popularity_? It turns out that there can be different things justifying the popularity of content:
- **pure count**: simply count the number of ratings or interactions an item received regardless of their quality
- **positive count**: only count the number of ratings or interactions that we assume reflect preference towards items, e.g. ratings above user mean ratings
- **time-dependency**: despite evergreen stars items may also be popular for a limited time only - how can we account for this?

**Remark**: Popularity ranking entails no personalization. We obtain a single popularity ranking of items which is independent from the user and serve the same top-$N$ items to every user.

### Popularity based on simple Interaction Counts

![](../parrot.png)

**Task**: Infer the item popularity order from training ratings as an array with items in descending order of popularity.

In [10]:
item_popularity = data.train_ratings['item'].value_counts()

In [11]:
item_popularity

50      471
181     423
258     409
100     405
294     394
       ... 
1320      1
1669      1
1576      1
1541      1
1663      1
Name: item, Length: 1651, dtype: int64

In [12]:
item_order = item_popularity.index.values

In [13]:
item_order

array([  50,  181,  258, ..., 1576, 1541, 1663])

What are the most popular movies?

In [14]:
top_movie_ids = item_order[:5]
items[items['item'].isin(top_movie_ids)][['item', 'title']]

,item,title
49,50,Star Wars (1977)
99,100,Fargo (1996)
180,181,Return of the Jedi (1983)
257,258,Contact (1997)
293,294,Liar Liar (1997)


### Popularity based on positive Interaction Counts

We assume that the the mean rating for each user is the threshold above which movies are regarded as favorable and below which movies are deemed as bad.

1. compute that user mean rating for each user.
2. remove all ratings that fall below this threshold.
3. apply the process above to the remaining ratings.

In [15]:
user_mean_ratings = data.train_ratings[['user', 'rating']].groupby('user')
user_mean_ratings = user_mean_ratings.mean().reset_index()
user_mean_ratings.rename(columns={'rating': 'user_mean_rating'},
                         inplace=True)

In [16]:
user_mean_ratings

,user,user_mean_rating
0,1,3.590476
1,2,3.673469
2,3,2.809524
3,4,4.333333
4,5,2.925373
...,...,...
938,939,4.292683
939,940,3.425287
940,941,3.947368
941,942,4.234375


In [17]:
positive_train_ratings = data.train_ratings.merge(user_mean_ratings,
                                                  on='user',
                                                  how='left')

In [18]:
keep_ratings = (positive_train_ratings['rating'] >= positive_train_ratings['user_mean_rating'])

In [19]:
positive_train_ratings = positive_train_ratings[keep_ratings]
positive_train_ratings.drop(columns='user_mean_rating', inplace=True)

In [20]:
positive_train_ratings

,user,item,rating,timestamp
0,877,381,4,882677345
2,94,431,4,891721716
6,598,286,5,886711452
7,886,496,4,876031952
9,521,184,4,884478358
...,...,...,...,...
79989,200,673,5,884128554
79993,44,432,5,878347569
79995,92,48,4,875653307
79997,1,96,5,875072716


In [21]:
item_popularity_positive = positive_train_ratings.item.value_counts()

In [22]:
item_popularity_positive

50      395
100     319
181     303
174     278
127     273
       ... 
1534      1
1598      1
1630      1
1662      1
1631      1
Name: item, Length: 1451, dtype: int64

In [23]:
item_order_positive = item_popularity.index.values

Again, let's have a look at the top movies in that order:

In [24]:
top_movie_ids = item_order_positive[:5]
items[items['item'].isin(top_movie_ids)][['item', 'title']]

,item,title
49,50,Star Wars (1977)
99,100,Fargo (1996)
180,181,Return of the Jedi (1983)
257,258,Contact (1997)
293,294,Liar Liar (1997)


#### How strong do both orderings correlate with each other?

Check spearman rank correlation between both orderings to quantify the distortion in ordering.

In [25]:
joint_counts = [[item_popularity.loc[item], item_popularity_positive[item]]
                for item in np.intersect1d(item_popularity_positive.index.values,
                                           item_popularity.index.values)]
joint_counts = np.array(joint_counts)

In [26]:
joint_counts

array([[365, 252],
       [105,  40],
       [ 72,  28],
       ...,
       [  4,   3],
       [  1,   1],
       [  1,   1]])

In [27]:
spearmanr(joint_counts)

SpearmanrResult(correlation=0.946241194276375, pvalue=0.0)

### Using Popularity Ordering for top-$N$ Recommendations

In [28]:
item_order

array([  50,  181,  258, ..., 1576, 1541, 1663])

In [29]:
item_order_positive

array([  50,  181,  258, ..., 1576, 1541, 1663])

![](../parrot.png)

**Task**: Write a method `get_recommendation` that returns the top-$N$ items without any known positives, i.e. items the user has already viewed.

In [30]:
def get_recommendations(user: int,
                        user_ratings: dict,
                        item_popularity_order: np.array,
                        N: int) -> List[int]:
    known_positives = list(user_ratings[user].keys())
    recommendations = np.setdiff1d(item_popularity_order,
                                   known_positives)[:N]
    
    return recommendations

Try it ...

In [31]:
get_recommendations(1, user_ratings, item_order, 10)

array([ 2,  5,  9, 13, 15, 17, 27, 28, 34, 39])

## Evaluating the Relevance of Recommendations

In [32]:
def get_relevant_items(test_ratings: pd.DataFrame) -> Dict[int, List[int]]:
    """
    returns {user: [items]} as a list of relevant items per user
    for all users found in the test dataset
    """
    relevant_items = test_ratings[['user', 'item']]
    relevant_items = relevant_items.groupby('user')
    relevant_items = {user: relevant_items.get_group(user)['item'].values
                      for user in relevant_items.groups.keys()}

    return relevant_items

In [33]:
relevant_items = get_relevant_items(data.test_ratings)

In [34]:
relevant_items[1]

array([143,  88, 131,  15, 239,  99, 255, 134, 210, 264, 227, 202,  59,
        17, 203,   9,  39,  55, 251, 213,  27, 137, 180, 207, 196, 101,
        28, 123,  13, 268, 106, 112, 222,  93, 126, 246, 257,  98,  34,
       130, 218, 188, 242, 270, 170, 122, 120, 224,  72, 183, 190, 146,
       167, 214, 121,   2,   5, 182,  43, 175, 252, 184])

### $Precision@10$

Now, we can compute the intersection between the top-$N$ recommended items and the items each user interacted with. Ideally, we want every recommendation to be a hit, i.e. an item the user consumed. In this case the size of intersections is $N$ given $N$ recommendations which is a precision of 100% = $\frac{N}{N}$.

We compute the so called $Precision@N$ for every user and take the mean over all. The resulting metric is called _mean average precision at N_ or short $MAP@N$.

![](../parrot.png)

**Task:** Compute the $MAP@N$ for popularity recommendations

In [36]:
def get_precision(users: List[int], user_ratings: Dict[int, Dict[int, float]],
                  item_order: np.array, N: int) -> Dict[int, float]:
    
    prec_at_N = dict.fromkeys(users)
    
    for user in users:
        recommendations = get_recommendations(user,
                                              user_ratings,
                                              item_order,
                                              N)
        hits = np.intersect1d(recommendations,
                              relevant_items[user])
        prec_at_N[user] = len(hits)/N
    
    return prec_at_N

In [35]:
N = 10
users = relevant_items.keys()

In [37]:
prec_at_N = get_precision(users, user_ratings, item_order, N)

In [38]:
np.mean(list(prec_at_N.values()))

0.06212765957446809